
# INSTALLATION

In [1]:
!pip install aif360
!pip install fairlearn

In [2]:
!apt-get install -jre
!java -version

E: Command line option 'j' [from -jre] is not understood in combination with the other options.
openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [3]:
!pip install h2o

In [4]:
!pip install xlsxwriter

In [5]:
!pip install BlackBoxAuditing

#IMPORTS

In [6]:
import numpy as np
from mlxtend.feature_selection import  ExhaustiveFeatureSelector
from xgboost import  XGBClassifier
# import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter
from openpyxl import  load_workbook

import BlackBoxAuditing
import shap
#suppress setwith copy warning
pd.set_option('mode.chained_assignment',None)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectFwe, SelectPercentile,SelectFdr, SelectFpr, SelectFromModel
from sklearn.feature_selection import chi2, mutual_info_classif
# from skfeature.function.similarity_based import fisher_score
from aif360.algorithms.inprocessing import PrejudiceRemover
import matplotlib.pyplot as plt
from aif360.metrics.classification_metric import ClassificationMetric

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing, LFR,OptimPreproc
from aif360.datasets import StandardDataset , BinaryLabelDataset
from sklearn.preprocessing import MinMaxScaler 
MM= MinMaxScaler()
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

import sys
sys.path.append("../")
import os



/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5plbzhsw
  JVM stdout: /tmp/tmp5plbzhsw/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp5plbzhsw/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,2 days
H2O_cluster_name:,H2O_from_python_unknownUser_el47h8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


#**************************LOADING DATASET*******************************

In [21]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# PR REMOVER 1


In [25]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = train.pop('PROGRAM_ENROLMENT_STATUS')
  train.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = test.pop('PROGRAM_ENROLMENT_STATUS')
  test.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  
  #******************************** prejudice remover regularizer*****************************
  Classifier = PrejudiceRemover(eta= 1, sensitive_attr= 'HOME_LANGUAGE')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['PROGRAM_ENROLMENT_STATUS']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'HOME_LANGUAGE':1}]
  disadvantagedGroup= [{'HOME_LANGUAGE':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover1.xlsx')
  Dropout= excelBook['Dropout']
  data= Dropout.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover1.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Dropout', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.7122302158273381
Accuracy 0.7338129496402878
Accuracy 0.6811594202898551
Accuracy 0.7536231884057971
Accuracy 0.7608695652173914
Accuracy 0.7194244604316546
Accuracy 0.7266187050359713
Accuracy 0.7391304347826086
Accuracy 0.782608695652174
Accuracy 0.717391304347826
Accuracy 0.762589928057554
Accuracy 0.7769784172661871
Accuracy 0.7246376811594203
Accuracy 0.6956521739130435
Accuracy 0.6811594202898551
Accuracy 0.7553956834532374
Accuracy 0.7338129496402878
Accuracy 0.6521739130434783
Accuracy 0.717391304347826
Accuracy 0.7318840579710145
Accuracy 0.7266187050359713
Accuracy 0.7122302158273381
Accuracy 0.7101449275362319
Accuracy 0.717391304347826
Accuracy 0.7608695652173914
Accuracy 0.7338129496402878
Accuracy 0.7194244604316546
Accuracy 0.7246376811594203
Accuracy 0.7101449275362319
Accuracy 0.717391304347826
Accuracy 0.6906474820143885
Accuracy 0.7697841726618705
Accuracy 0.7246376811594203
Accuracy 0.7246376811594203
Accuracy 0.7246376811594203
Accuracy 0.762589928057554

# PR REMOVER 1


In [ ]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = train.pop('PROGRAM_ENROLMENT_STATUS')
  train.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = test.pop('PROGRAM_ENROLMENT_STATUS')
  test.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  
  #******************************** prejudice remover regularizer*****************************
  Classifier = PrejudiceRemover(eta= 25, sensitive_attr= 'HOME_LANGUAGE')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['PROGRAM_ENROLMENT_STATUS']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'HOME_LANGUAGE':1}]
  disadvantagedGroup= [{'HOME_LANGUAGE':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover1.xlsx')
  Dropout= excelBook['Dropout']
  data= Dropout.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover1.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Dropout', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.7122302158273381
Accuracy 0.7338129496402878
Accuracy 0.6811594202898551
Accuracy 0.7536231884057971
Accuracy 0.7608695652173914
Accuracy 0.7194244604316546
Accuracy 0.7266187050359713
Accuracy 0.7391304347826086
Accuracy 0.782608695652174
Accuracy 0.717391304347826
Accuracy 0.762589928057554
Accuracy 0.7769784172661871
Accuracy 0.7246376811594203
Accuracy 0.6956521739130435
Accuracy 0.6811594202898551
Accuracy 0.7553956834532374
Accuracy 0.7338129496402878
Accuracy 0.6521739130434783
Accuracy 0.717391304347826
Accuracy 0.7318840579710145
Accuracy 0.7266187050359713
Accuracy 0.7122302158273381
Accuracy 0.7101449275362319
Accuracy 0.717391304347826
Accuracy 0.7608695652173914
Accuracy 0.7338129496402878
Accuracy 0.7194244604316546
Accuracy 0.7246376811594203
Accuracy 0.7101449275362319
Accuracy 0.717391304347826
Accuracy 0.6906474820143885
Accuracy 0.7697841726618705
Accuracy 0.7246376811594203
Accuracy 0.7246376811594203
Accuracy 0.7246376811594203
Accuracy 0.762589928057554

# PR REMOVER 25


In [9]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = train.pop('PROGRAM_ENROLMENT_STATUS')
  train.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = test.pop('PROGRAM_ENROLMENT_STATUS')
  test.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  
  #******************************** prejudice remover regularizer*****************************
  Classifier = PrejudiceRemover(eta= 25, sensitive_attr= 'HOME_LANGUAGE')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['PROGRAM_ENROLMENT_STATUS']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'HOME_LANGUAGE':1}]
  disadvantagedGroup= [{'HOME_LANGUAGE':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover25.xlsx')
  Dropout= excelBook['Dropout']
  data= Dropout.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover25.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Dropout', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.7122302158273381
Accuracy 0.7338129496402878
Accuracy 0.6811594202898551
Accuracy 0.7536231884057971
Accuracy 0.7608695652173914
Accuracy 0.7194244604316546
Accuracy 0.7266187050359713
Accuracy 0.7391304347826086
Accuracy 0.7608695652173914
Accuracy 0.717391304347826
Accuracy 0.762589928057554
Accuracy 0.7769784172661871
Accuracy 0.7246376811594203
Accuracy 0.6956521739130435
Accuracy 0.6811594202898551
Accuracy 0.7553956834532374
Accuracy 0.7338129496402878
Accuracy 0.6521739130434783
Accuracy 0.717391304347826
Accuracy 0.7318840579710145
Accuracy 0.7194244604316546
Accuracy 0.7122302158273381
Accuracy 0.7101449275362319
Accuracy 0.7246376811594203
Accuracy 0.7608695652173914
Accuracy 0.7338129496402878
Accuracy 0.7194244604316546
Accuracy 0.7246376811594203
Accuracy 0.7101449275362319
Accuracy 0.7246376811594203
Accuracy 0.6906474820143885
Accuracy 0.7697841726618705
Accuracy 0.7246376811594203
Accuracy 0.7246376811594203
Accuracy 0.717391304347826
Accuracy 0.7625899280575

# PR REMOVER 50



In [17]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = train.pop('PROGRAM_ENROLMENT_STATUS')
  train.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = test.pop('PROGRAM_ENROLMENT_STATUS')
  test.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  
  #******************************** prejudice remover regularizer*****************************
  Classifier = PrejudiceRemover(eta= 50, sensitive_attr= 'HOME_LANGUAGE')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['PROGRAM_ENROLMENT_STATUS']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'HOME_LANGUAGE':1}]
  disadvantagedGroup= [{'HOME_LANGUAGE':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover50.xlsx')
  Dropout= excelBook['Dropout']
  data= Dropout.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover50.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Dropout', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.7122302158273381
Accuracy 0.7338129496402878
Accuracy 0.6811594202898551
Accuracy 0.7536231884057971
Accuracy 0.7608695652173914
Accuracy 0.7194244604316546
Accuracy 0.7266187050359713
Accuracy 0.7391304347826086
Accuracy 0.7536231884057971
Accuracy 0.717391304347826
Accuracy 0.762589928057554
Accuracy 0.7769784172661871
Accuracy 0.7246376811594203
Accuracy 0.6956521739130435
Accuracy 0.6811594202898551
Accuracy 0.7553956834532374
Accuracy 0.7338129496402878
Accuracy 0.6521739130434783
Accuracy 0.717391304347826
Accuracy 0.7318840579710145
Accuracy 0.7194244604316546
Accuracy 0.7122302158273381
Accuracy 0.7101449275362319
Accuracy 0.7246376811594203
Accuracy 0.7608695652173914
Accuracy 0.6906474820143885
Accuracy 0.7194244604316546
Accuracy 0.7246376811594203
Accuracy 0.7101449275362319
Accuracy 0.7246376811594203
Accuracy 0.6906474820143885
Accuracy 0.7697841726618705
Accuracy 0.7246376811594203
Accuracy 0.7246376811594203
Accuracy 0.717391304347826
Accuracy 0.7625899280575

# PR REMOVER 75



In [18]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = train.pop('PROGRAM_ENROLMENT_STATUS')
  train.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = test.pop('PROGRAM_ENROLMENT_STATUS')
  test.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  
  #******************************** prejudice remover regularizer*****************************
  Classifier = PrejudiceRemover(eta= 75, sensitive_attr= 'HOME_LANGUAGE')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['PROGRAM_ENROLMENT_STATUS']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'HOME_LANGUAGE':1}]
  disadvantagedGroup= [{'HOME_LANGUAGE':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover75.xlsx')
  Dropout= excelBook['Dropout']
  data= Dropout.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover75.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Dropout', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.7122302158273381
Accuracy 0.7338129496402878
Accuracy 0.6811594202898551
Accuracy 0.7536231884057971
Accuracy 0.7608695652173914
Accuracy 0.7194244604316546
Accuracy 0.7266187050359713
Accuracy 0.7391304347826086
Accuracy 0.7536231884057971
Accuracy 0.717391304347826
Accuracy 0.762589928057554
Accuracy 0.7769784172661871
Accuracy 0.7246376811594203
Accuracy 0.6956521739130435
Accuracy 0.6811594202898551
Accuracy 0.7553956834532374
Accuracy 0.7338129496402878
Accuracy 0.6521739130434783
Accuracy 0.717391304347826
Accuracy 0.7318840579710145
Accuracy 0.7194244604316546
Accuracy 0.7122302158273381
Accuracy 0.7101449275362319
Accuracy 0.7246376811594203
Accuracy 0.7608695652173914
Accuracy 0.6618705035971223
Accuracy 0.7194244604316546
Accuracy 0.7246376811594203
Accuracy 0.7101449275362319
Accuracy 0.7246376811594203
Accuracy 0.6906474820143885
Accuracy 0.7697841726618705
Accuracy 0.7246376811594203
Accuracy 0.7246376811594203
Accuracy 0.717391304347826
Accuracy 0.7625899280575

# PR REMOVER 100



In [22]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = train.pop('PROGRAM_ENROLMENT_STATUS')
  train.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Dropout/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['STUDENT_KEY','CAREER_TERM_YEAR'],axis=1)
  first_column = test.pop('PROGRAM_ENROLMENT_STATUS')
  test.insert(0, 'PROGRAM_ENROLMENT_STATUS', first_column)


  # # normalization of train and test sets
  # Fitter= MM.fit(train)
  # transformed_train=Fitter.transform(train)
  # train=pd.DataFrame(transformed_train, columns= train.columns)

  # #test normalization
  # transformed_test=Fitter.transform(test)
  # test=pd.DataFrame(transformed_test, columns= test.columns)


  ## ****************CONVERTING TO BLD FORMAT******************************
  #BLD Train set
  class Train(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )
  # BLD Test
  BLD_Train = Train(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  

  class Test(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])
  
  #******************************** prejudice remover regularizer*****************************
  Classifier = PrejudiceRemover(eta= 100, sensitive_attr= 'HOME_LANGUAGE')
  Classifier.fit(BLD_Train)
  prediction= Classifier.predict(BLD_Test)
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  # gbm_Predictions= best_model.predict(Test)
  # gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['PROGRAM_ENROLMENT_STATUS']= prediction.labels

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'HOME_LANGUAGE':1}]
  disadvantagedGroup= [{'HOME_LANGUAGE':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'PROGRAM_ENROLMENT_STATUS',
                  favorable_classes= [1],protected_attribute_names=['HOME_LANGUAGE'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['HOME_LANGUAGE'],
                        privileged_classes= [[1]])


  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover100.xlsx')
  Dropout= excelBook['Dropout']
  data= Dropout.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/PRemover/PRemover100.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Dropout', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Accuracy 0.7122302158273381
Accuracy 0.7338129496402878
Accuracy 0.6811594202898551
Accuracy 0.7536231884057971
Accuracy 0.7608695652173914
Accuracy 0.7194244604316546
Accuracy 0.7266187050359713
Accuracy 0.7391304347826086
Accuracy 0.7463768115942029
Accuracy 0.717391304347826
Accuracy 0.762589928057554
Accuracy 0.7769784172661871
Accuracy 0.7246376811594203
Accuracy 0.6956521739130435
Accuracy 0.6811594202898551
Accuracy 0.7553956834532374
Accuracy 0.7338129496402878
Accuracy 0.6521739130434783
Accuracy 0.717391304347826
Accuracy 0.7318840579710145
Accuracy 0.7194244604316546
Accuracy 0.7122302158273381
Accuracy 0.7101449275362319
Accuracy 0.7246376811594203
Accuracy 0.7608695652173914
Accuracy 0.6762589928057554
Accuracy 0.7194244604316546
Accuracy 0.7246376811594203
Accuracy 0.7101449275362319
Accuracy 0.7246376811594203
Accuracy 0.6906474820143885
Accuracy 0.7697841726618705
Accuracy 0.7246376811594203
Accuracy 0.7246376811594203
Accuracy 0.7028985507246377
Accuracy 0.762589928057